In [2]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 10936



In [3]:
tbl_name = f"sashelp.cars"
libref, tbl = tbl_name.split('.')

### confirm table exists

In [9]:
sas.exist(tbl, libref)

True

In [6]:
sas.submitLST(
f"""
%macro checkds(dsn);
   %if %sysfunc(exist(&dsn)) %then
      %do;
         proc print data=&dsn (obs=1);
         run;
      %end;
      %else
         %put The data set &dsn does not exist.;
%mend checkds;

%checkds({tbl_name})

""", method="listorlog")

Obs,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
1,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6,265,17,23,4451,106,189


In [7]:
sas.submitLST(
f"""
%checkds({tbl_name}ddd)
""", method="listandlog")


11                                                         The SAS System                         Thursday, May 30, 2024 11:16:00 AM

94         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
94       !  graphics on / outputfmt=png;
95         
96         
97         %checkds(sashelp.carsddd)
The data set sashelp.carsddd does not exist.
98         
99         
100        
101        ods html5 (id=saspy_internal) close;ods listing;
102        

12                                                         The SAS System                         Thursday, May 30, 2024 11:16:00 AM

103        


### head

In [3]:
cols = """
cylinders
drivetrain
enginesize
horsepower
length
mpg_city
mpg_highway
msrp
make
model
origin
type
weight
wheelbase
"""

cols = cols.strip().split('\n')
cols

['cylinders',
 'drivetrain',
 'enginesize',
 'horsepower',
 'length',
 'mpg_city',
 'mpg_highway',
 'msrp',
 'make',
 'model',
 'origin',
 'type',
 'weight',
 'wheelbase']

In [4]:
sdf = sas.sasdata(
    tbl,
    libref,
    dsopts={
        'keep' : ' '.join(cols),
    }
)
df = sdf.head()
df.columns = df.columns.str.lower()

df

,make,model,type,origin,drivetrain,msrp,enginesize,cylinders,horsepower,mpg_city,mpg_highway,weight,wheelbase,length
0,Acura,MDX,SUV,Asia,All,36945.0,3.5,6.0,265.0,17.0,23.0,4451.0,106.0,189.0
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,23820.0,2.0,4.0,200.0,24.0,31.0,2778.0,101.0,172.0
2,Acura,TSX 4dr,Sedan,Asia,Front,26990.0,2.4,4.0,200.0,22.0,29.0,3230.0,105.0,183.0
3,Acura,TL 4dr,Sedan,Asia,Front,33195.0,3.2,6.0,270.0,20.0,28.0,3575.0,108.0,186.0
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,43755.0,3.5,6.0,225.0,18.0,24.0,3880.0,115.0,197.0


In [5]:
print(df.iloc[0,:].to_string())

make             Acura
model              MDX
type               SUV
origin            Asia
drivetrain         All
msrp           36945.0
enginesize         3.5
cylinders          6.0
horsepower       265.0
mpg_city          17.0
mpg_highway       23.0
weight          4451.0
wheelbase        106.0
length           189.0
Name: 0, dtype: object

### dim

In [4]:
sas.sasdata(tbl, libref).obs()

428

In [9]:
sas.sasdata(tbl, libref).info().shape[0]

15

In [ ]:
sas.submitLST(
f"""
%macro obsnvars(ds);
   %global dset nvars nobs;
   %let dset=&ds;
   %let dsid = %sysfunc(open(&dset));
   %if &dsid %then
      %do;
         %let nobs =%sysfunc(attrn(&dsid,NOBS));
         %let nvars=%sysfunc(attrn(&dsid,NVARS));
         %let rc = %sysfunc(close(&dsid));
         %put &nobs, &nvars;
      %end;
   %else
      %put Open for data set &dset failed - %sysfunc(sysmsg());
%mend obsnvars;

%obsnvars({tbl_name});

""", method="listorlog")


15                                                         The SAS System                         Thursday, May 30, 2024 11:53:00 AM

94         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=Plateau; ods
94       !  graphics on / outputfmt=png;
95         
96         
97         %macro obsnvars(ds);
98            %global dset nvars nobs;
99            %let dset=&ds;
100           %let dsid = %sysfunc(open(&dset));
101           %if &dsid %then
102              %do;
103                 %let nobs =%sysfunc(attrn(&dsid,NOBS));
104                 %let nvars=%sysfunc(attrn(&dsid,NVARS));
105                 %let rc = %sysfunc(close(&dsid));
106                 %put &dset has &nvars  variable(s) and &nobs observation(s).;
107              %end;
108           %else
109              %put Open for data set &dset failed - %sysfunc(sysmsg());
110        %mend obsnvars;
111        
112        %obsnvars(sashelp.cars);
sashelp.cars has

### metadata

In [6]:
sdf = sas.sasdata(tbl, libref=libref)
df = sdf.columnInfo()
df = df[df['Variable'].str.lower().isin(cols)]
df

,Member,Num,Variable,Type,Len,Pos,Format,Label
0,SASHELP.CARS,9.0,Cylinders,Num,8.0,24.0,NaN,NaN
1,SASHELP.CARS,5.0,DriveTrain,Char,5.0,147.0,NaN,NaN
2,SASHELP.CARS,8.0,EngineSize,Num,8.0,16.0,NaN,Engine Size (L)
3,SASHELP.CARS,10.0,Horsepower,Num,8.0,32.0,NaN,NaN
5,SASHELP.CARS,15.0,Length,Num,8.0,72.0,NaN,Length (IN)
6,SASHELP.CARS,11.0,MPG_City,Num,8.0,40.0,NaN,MPG (City)
7,SASHELP.CARS,12.0,MPG_Highway,Num,8.0,48.0,NaN,MPG (Highway)
8,SASHELP.CARS,6.0,MSRP,Num,8.0,0.0,DOLLAR8.,NaN
9,SASHELP.CARS,1.0,Make,Char,13.0,80.0,NaN,NaN
10,SASHELP.CARS,2.0,Model,Char,40.0,93.0,NaN,NaN


In [7]:
sas.submitLST(
f"""
proc sql;
    create view {tbl}_sub as
    select {', '.join(cols)}
    from {tbl_name}
    where 1=1;
quit;

PROC MEANS DATA={tbl}_sub;
RUN;

""", method="listorlog")


Variable,Label,N,Mean,Std Dev,Minimum,Maximum
Cylinders EngineSize Horsepower Length MPG_City MPG_Highway MSRP Weight Wheelbase,Engine Size (L) Length (IN) MPG (City) MPG (Highway) Weight (LBS) Wheelbase (IN),426 428 428 428 428 428 428 428 428,5.8075117 3.1967290 215.8855140 186.3621495 20.0607477 26.8434579 32774.86 3577.95 108.1542056,1.5584426 1.1085947 71.8360316 14.3579913 5.2382176 5.7412007 19431.72 758.9832146 8.3118130,3.0000000 1.3000000 73.0000000 143.0000000 10.0000000 12.0000000 10280.00 1850.00 89.0000000,12.0000000 8.3000000 500.0000000 238.0000000 60.0000000 66.0000000 192465.00 7190.00 144.0000000
